### Can raw feature be discriminated for interictal and preictal classes?

In [1]:
import numpy as np
import mne
from tqdm import tqdm
from ARMA import ARMA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, auc, precision_recall_curve, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas.plotting import parallel_coordinates
import yasa

/Users/sameenislam/anaconda3/envs/ml/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.4.1, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
root = '/Volumes/My Passport/AI_Research/data/physionet.org/files/chbmit/1.0.0/' 
case = 'chb01/'
preictal_filenames = ['chb01_03_preictal.npy', 'chb01_04_preictal.npy', 'chb01_15_preictal.npy', 'chb01_16_preictal.npy',
             'chb01_18_preictal.npy', 'chb01_26_preictal.npy']
interictal_filenames = ['chb01_01.edf', 'chb01_02.edf', 'chb01_05.edf',
                        'chb01_06.edf', 'chb01_07.edf', 'chb01_08.edf']

In [3]:
def load_EEG(filepath, label) -> (np.ndarray, np.ndarray):
    if label == 'preictal':
        data = np.load(filepath)
        target = np.ones(data.shape[1])
    if label == 'interictal':
        data = mne.io \
        .read_raw_edf(input_fname=filepath, preload=False, verbose='Error') \
        .get_data(picks='all', units='uV', return_times=False)
        target = -1 * np.ones(data.shape[1])
    return data, target

In [5]:
# load preictal and interictal data
preictal_feature_list = []
interictal_feature_list = []
for i in tqdm(range(6)):
    # load preictal data
    p_filepath = root + case + preictal_filenames[i]
    p_data, _ = load_EEG(p_filepath, 'preictal')
    # load interictal data
    ic_filepath = root + case + interictal_filenames[i]
    ic_data, _ = load_EEG(ic_filepath, 'interictal')
    
    # store data in list
    preictal_feature_list.append(p_data)
    interictal_feature_list.append(ic_data)

100%|██████████| 6/6 [00:03<00:00,  1.74it/s]


In [19]:
df_1 = pd.DataFrame(preictal_feature_list[0].T)
df_1['Interval'] = 'Preictal'
df_2 = pd.DataFrame(interictal_feature_list[0].T)
df_2['Interval'] = 'Interictal'
frames = [df_1, df_2]
rdf = pd.concat(frames)

In [20]:
rdf

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,Interval
0,10.354090,0.586081,-7.619048,14.652015,9.181929,-5.665446,15.824176,-0.586081,7.619048,-7.619048,...,-20.122100,30.671551,-19.731380,13.870574,8.009768,1.367521,9.181929,6.056166,-20.122100,Preictal
1,12.698413,2.539683,-6.837607,15.433455,11.526252,-6.056166,16.605617,0.976801,12.307692,-8.400488,...,-14.652015,36.141636,-20.122100,16.214896,7.228327,-0.586081,3.711844,6.446886,-14.652015,Preictal
2,13.870574,6.446886,-9.181929,14.652015,14.261294,-6.056166,16.605617,0.976801,14.261294,-8.009768,...,-9.963370,36.532357,-21.294261,16.605617,9.572650,-6.837607,-2.539683,5.665446,-9.963370,Preictal
3,15.824176,7.228327,-11.526252,15.824176,18.168498,-8.009768,13.870574,3.711844,17.777778,-8.009768,...,-6.837607,42.002442,-21.684982,15.824176,11.916972,-11.135531,-8.400488,4.493284,-6.837607,Preictal
4,18.559219,5.665446,-11.916972,15.042735,24.420024,-13.479853,11.135531,5.274725,18.559219,-7.228327,...,-5.274725,51.379731,-20.903541,13.479853,12.307692,-9.963370,-13.089133,0.976801,-5.274725,Preictal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921595,13.870574,11.135531,48.253968,6.056166,-7.619048,7.228327,38.876679,42.002442,-0.976801,7.619048,...,23.638584,-16.996337,16.605617,11.135531,-47.863248,-42.783883,32.234432,11.135531,23.638584,Interictal
921596,-1.758242,21.294261,54.114774,-10.354090,-25.201465,23.247863,49.816850,14.261294,-0.586081,9.181929,...,15.824176,-31.452991,28.717949,18.559219,-53.724054,-27.936508,31.843712,0.976801,15.824176,Interictal
921597,-11.526252,23.638584,48.644689,-17.777778,-34.969475,38.095238,40.048840,0.976801,-2.148962,1.758242,...,16.996337,-17.387057,24.810745,18.949939,-48.253968,-11.916972,0.976801,-7.228327,16.996337,Interictal
921598,-2.930403,27.545788,45.128205,-20.512821,-25.982906,48.644689,33.797314,-6.446886,-2.148962,2.930403,...,22.466422,-1.758242,28.717949,20.512821,-44.737485,-3.711844,-15.824176,-10.354090,22.466422,Interictal


In [ ]:
# takes too long to run; no results.
sns.pairplot(rdf, hue='Interval', height=1.8, aspect=1.8,
                  plot_kws=dict(edgecolor="k", linewidth=0.5),
                  diag_kind="scatter", diag_kws=dict(shade=True))
plt.show()